# ===== Guess The Number Game =====

## Guess the Number 🎮 (Single Player)

This is a simple Python game where the computer randomly picks a secret number within a chosen range.  
Your goal is to guess the number within a limited number of attempts.

- The game tells you if your guess is **too high** or **too low**.  
- You only have a fixed number of attempts to get it right.  
- If you guess correctly, you win 🎉. Otherwise, the game reveals the secret number at the end.  

It’s a fun beginner-friendly project that introduces:
- User input handling  
- Random number generation  
- Conditional logic  
- Loops and attempts tracking  


In [1]:
import random

def guess_the_number(low=1, high=10, attempts=3):
    print("\n====== Guess The Number Game =====")
    print(f"\nI'm thinking of a number between {low} and {high}.")
    print(f"\nYou have {attempts} attempts to guess it.\n")

    secret_number = random.randint(low, high)

    for attempt in range(1, attempts + 1):
        while True:
            try:
                guess = int(input(f"Attempt {attempt} - Enter your guess: "))
                break
            except ValueError:
                print("Invalid input. Please enter a number.")

        if guess == secret_number:
            print(f"\n🎉 Congratulations! You guessed right in {attempt} attempts!")
            return
        elif guess > secret_number:
            print("Too high! Try a lower number.")
        else:
            print("Too low! Try a higher number.")

        print(f"Attempts left: {attempts - attempt}\n")

    print("\n❌ Game Over!")
    print(f"The secret number was: {secret_number}")
    print("Better luck next time!\n")




In [2]:
# Run game
guess_the_number()


====== Guess The Number Game =====

I'm thinking of a number between 1 and 10.

You have 3 attempts to guess it.



Attempt 1 - Enter your guess:  7


Too high! Try a lower number.
Attempts left: 2



Attempt 2 - Enter your guess:  5


Too high! Try a lower number.
Attempts left: 1



Attempt 3 - Enter your guess:  2


Too low! Try a higher number.
Attempts left: 0


❌ Game Over!
The secret number was: 4
Better luck next time!



## Multiplayer Guess the Number 🎮

This is a pass-and-play Python game where multiple players compete to guess the same secret number.  
Each round, players take turns making guesses until someone gets it right or everyone runs out of attempts.

- The computer secretly picks a number within a chosen range.  
- Players take turns guessing.  
- The game gives hints if a guess is **too high** or **too low**, and sometimes if it’s **close**.  
- The first player to guess correctly wins the round and earns a point.  
- After all rounds are played, the leaderboard is displayed and the player with the most points wins.  

This version introduces:
- **Multiple players with scores and leaderboard tracking**  
- **Fair turn rotation across players**  
- **Configurable rounds, range, and attempts per player**  
- **Hints for “close” guesses**  

It’s a fun way to extend the classic single-player game into a competitive experience for friends or classmates.


In [3]:
import random
import math

def get_int(prompt, min_val=None, max_val=None):
    """Safe integer input with optional bounds."""
    while True:
        s = input(prompt).strip()
        if s.lstrip("-").isdigit():
            n = int(s)
            if (min_val is None or n >= min_val) and (max_val is None or n <= max_val):
                return n
        print(f"Please enter a valid integer"
              f"{' ≥ ' + str(min_val) if min_val is not None else ''}"
              f"{' and ≤ ' + str(max_val) if max_val is not None else ''}.")

def setup_players():
    """Collect unique, non-empty player names."""
    while True:
        try:
            count = get_int("How many players? (2–8): ", 2, 8)
            break
        except Exception:
            pass
    players = []
    for i in range(1, count + 1):
        while True:
            name = input(f"Player {i} name: ").strip()
            if name and name not in players:
                players.append(name)
                break
            print("Name must be non-empty and unique.")
    return players

def play_round(players, low, high, attempts_per_player, round_num):
    """Plays one round. Returns (winner_name or None, guess_log)."""
    secret = random.randint(low, high)
    guess_log = []  # (player, guess, relation)

    print(f"\n— Round {round_num} —")
    print(f"I'm thinking of a number between {low} and {high}.")
    total_turns = attempts_per_player * len(players)

    for turn in range(1, total_turns + 1):
        player = players[(turn - 1) % len(players)]
        guess = get_int(f"[{player}] Guess {turn}/{total_turns}: ", low, high)

        if guess == secret:
            print(f"✅ {player} guessed it! The number was {secret}.")
            guess_log.append((player, guess, "correct"))
            return player, guess_log
        elif guess < secret:
            print("Too low.")
            guess_log.append((player, guess, "low"))
        else:
            print("Too high.")
            guess_log.append((player, guess, "high"))

        # Optional “hot/cold” hint
        span = max(1, (high - low) // 10)
        if abs(guess - secret) <= span:
            print("Close!")

    print(f"❌ No one got it. The number was {secret}.")
    return None, guess_log

def print_leaderboard(scores):
    print("\n=== Leaderboard ===")
    # Sort by score desc, then name asc
    for name, pts in sorted(scores.items(), key=lambda x: (-x[1], x[0])):
        print(f"{name}: {pts}")

def multiplayer_guess_the_number():
    print("\n====== Guess The Number — Multiplayer ======\n")

    players = setup_players()
    print()
    low = get_int("Lower bound (e.g., 1): ")
    high = get_int("Upper bound (e.g., 100): ", min_val=low + 1)
    rounds = get_int("How many rounds? (e.g., 3): ", 1, 50)

    # Smart default attempts per player ~ log2(range)+1 (rounded up)
    default_attempts = math.ceil(math.log2(high - low + 1)) + 1
    attempts_per_player = get_int(
        f"Attempts per player per round (default {default_attempts}): ",
        1, 20
    )
    # If user typed the default explicitly, keep; otherwise, they chose a number.

    scores = {p: 0 for p in players}

    for r in range(1, rounds + 1):
        winner, _ = play_round(players, low, high, attempts_per_player, r)
        if winner:
            scores[winner] += 1
        print_leaderboard(scores)

    # Final results
    max_pts = max(scores.values())
    champs = [p for p, s in scores.items() if s == max_pts]

    print("\n=== Final Result ===")
    print_leaderboard(scores)
    if len(champs) == 1:
        print(f"\n🏆 Winner: {champs[0]} with {max_pts} point(s)!")
    else:
        print(f"\n🤝 It's a tie between: {', '.join(champs)} with {max_pts} point(s)!")




In [4]:
# Run game
multiplayer_guess_the_number()


====== Guess The Number — Multiplayer ======



How many players? (2–8):  3
Player 1 name:  Williams
Player 2 name:  Amaka
Player 3 name:  Fiona


Lower bound (e.g., 1):  1
Upper bound (e.g., 100):  20
How many rounds? (e.g., 3):  2
Attempts per player per round (default 6):  3



— Round 1 —
I'm thinking of a number between 1 and 20.


[Williams] Guess 1/9:  5


...

Too high.


[Williams] Guess 7/9:  3


Too high.


[Amaka] Guess 8/9:  1


✅ Amaka guessed it! The number was 1.

=== Leaderboard ===
Amaka: 1
Fiona: 0
Williams: 0

=== Final Result ===

=== Leaderboard ===
Amaka: 1
Fiona: 0
Williams: 0

🏆 Winner: Amaka with 1 point(s)!
